<a href="https://colab.research.google.com/github/DavideRutigliano/frigate-models/blob/main/notebooks/YOLOv8_Pretrained_Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [2]:
%%capture
!pip install ultralytics

  Using cached ultralytics-8.3.90-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.m

## Select model

In [20]:
from ultralytics import YOLO

model_name = "yolo11s" #@param ["yolov8s", "yolov8m", "yolov8l", "yolo11s", "yolo11m", "yolo11l"]
input_width = 320 #@param {type:"slider", min:320, max:640, step:320}
input_height = 320 #@param {type:"slider", min:320, max:640, step:320}
quantization = "fp16"

MODEL_FILENAME = f"{model_name}_{quantization}_{input_width}x{input_height}.onnx"

In [21]:
model = YOLO(f"{model_name}.pt")
model.export(
  format="onnx",
  imgsz=[input_height,input_width],
  half=(quantization=="fp16"),
  optimize=False,
)
!mv {model_name}.onnx {MODEL_FILENAME}

100%|██████████| 18.4M/18.4M [00:00<00:00, 453MB/s]

Ultralytics 8.3.90 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0


YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs

PyTorch: starting from 'yolo11s.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 84, 2100) (18.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 1.9s, saved as 'yolo11s.onnx' (36.1 MB)

Export complete (2.5s)
Results saved to /content
Predict:         yolo predict task=detect model=yolo11s.onnx imgsz=320  
Validate:        yolo val task=detect model=yolo11s.onnx imgsz=320 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


In [22]:
import onnxruntime as ort
import numpy as np

dummy_input = np.random.randint(0, 255, (1, 3, input_width, input_height)).astype(np.float32)

ort_session = ort.InferenceSession(MODEL_FILENAME, providers=["ROCMExecutionProvider"])
ort_session.run(None, {"images": dummy_input})

[array([[[     9.8009,       10.41,      18.217, ...,       190.5,      202.91,       214.2],
         [     1.5174,      2.2267,       2.848, ...,      264.09,      268.32,      276.45],
         [     20.665,      19.563,      29.379, ...,      267.04,      237.23,      217.97],
         ...,
         [          0,           0,           0, ...,  4.4405e-06,  6.3181e-06,  3.5763e-07],
         [          0,           0,           0, ...,  2.1458e-06,  3.4571e-06,  1.7881e-06],
         [          0,           0,           0, ...,  1.0967e-05,  1.6749e-05,   2.861e-06]]], dtype=float32)]

## Download model

In [23]:
%cd /content
from google.colab import files

files.download(MODEL_FILENAME)

/content


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>